# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
import numpy as np
import pandas as pd
import zipfile
import sys
import requests
import nltk
from nltk.sentiment import SentimentAnalyzer
sa = SentimentAnalyzer()
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
sid = SentimentIntensityAnalyzer()
from selenium.webdriver import Firefox
import random
import time
import warnings
import pymongo
from selenium.common.exceptions import TimeoutException
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator
from pyspark.sql.types import (
    IntegerType, StringType, IntegerType, FloatType, 
    StructField, StructType, DoubleType)
from gc_functions import (get_keys, launch_spotipy, scrape_explore_page, build_most_popular, 
                          build_highest_rated, combine_and_remove_duplicates, print_sentiments, 
                          extract_user_comments, get_all_comments, get_tab_idxs, new_user_predict, 
                          check_db_size, sleep, make_url_list, get_data, get_comments, load_commentlist, 
                          store_commentlist, scrape_comments, download_mongodb)

# Initialize Mongodb, Spotipy

In [2]:
mongostring, myclientid, myclientsecret = get_keys()
mc = pymongo.MongoClient(mongostring)
sp = launch_spotipy(myclientid, myclientsecret)

# Build tables for most popular and highest rated tabs

In [3]:
#ratedtable = build_highest_rated()
#ratedtable.shape
#ratedtable.to_csv('ratedtable.csv', sep='~')
ratedtable = pd.read_csv('ratedtable.csv', sep='~', usecols = ['artist_name', 'artist_url', 'date', 'id', 'part', 'preset_id',
       'rating', 'recording', 'song_name', 'status', 'tab_access_type',
       'tab_url', 'tonality_name', 'tp_version', 'type', 'type_name',
       'verified', 'version', 'version_description', 'votes'])

In [4]:
#poptable = build_most_popular()
#poptable.shape
#poptable.to_csv('poptable.csv', sep='~')
poptable = pd.read_csv('poptable.csv', sep='~', usecols = ['artist_name', 'artist_url', 'date', 'id', 'part', 'preset_id',
       'rating', 'recording', 'song_name', 'status', 'tab_access_type',
       'tab_url', 'tonality_name', 'tp_version', 'type', 'type_name',
       'verified', 'version', 'version_description', 'votes'])

# Combine tables and extract urllist

In [5]:
combinedtable = combine_and_remove_duplicates(poptable, ratedtable)
combinedtable.shape

(1429, 20)

In [6]:
urllist = make_url_list(combinedtable)

# Autoscraping

In [7]:
#get_data(urllist, mc)

# Check Database Size

In [8]:
check_db_size(mc)

470

# Download Database

In [9]:
mydb = download_mongodb(mc)
len(mydb)

470

# Comment Extraction, Ratings List

In [10]:
mydb[0]['url']

['https://tabs.ultimate-guitar.com/tab/jeff_buckley/hallelujah_chords_198052',
 'Hallelujah']

In [16]:
ratingslist = extract_user_comments(mydb)
len(ratingslist)

28620

# Ratings Dataframe

In [19]:
ratingsdf = pd.DataFrame(ratingslist, columns = ['user', 'tab', 'rating'])
ratingsdf.shape

(28620, 3)